# 2.09: Prediction Intervals 101 + Conformal Prediction

## OPENING: WHY POINT FORECASTS FAIL THE BOARDROOM

We've built baseline models, computed metrics, built scoreboards, run diagnostics. We've measured accuracy. We've identified bias. We've found smoking guns.

But here's the problem: **We've only been looking at point forecasts.**

A point forecast is a single number. "Next week, we'll sell 100 units."

Great. Now the boardroom asks: **"How confident are you?"**

"What's the downside risk?"

"What happens if you're wrong by 20%?"

"How much safety stock do we need?"

A point forecast can't answer any of these questions.

**Intervals are the risk product.** They're the guardrails. They tell you: "Here's the likely range. Here's how much buffer you need."

## SETUP: Load Dependencies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

sns.set_theme()
plt.rcParams['figure.figsize'] = (14, 6)
pd.set_option('display.max_columns', None)

In [ ]:
# Load forecast data from cross-validation
# Expected columns: date, actual, forecast, residual, model_name, fold_id
cv_forecasts = pd.read_csv('path/to/cv_forecasts.csv')

print(f"Loaded {len(cv_forecasts)} forecast records")
print(f"Columns: {cv_forecasts.columns.tolist()}")

---
## SECTION 1: PREDICTION INTERVALS FUNDAMENTALS

### What Are Prediction Intervals?

**Point forecast:** "We'll sell 100 units next week"

**Prediction interval:** "We're 80% confident demand will be between 75 and 130 units"

The interval quantifies uncertainty. It tells you how wrong you might be.

### Two Jobs of a Prediction Interval

**Job 1: Validity (Calibration)**
- Does the interval hit its coverage level?
- If we ask for 80%, do we actually get ~80% coverage?

**Job 2: Efficiency (Usefulness)**
- Is the interval narrow enough to be operationally useful?
- Or is it so wide it's meaningless?

---
## SECTION 2: MODEL-BASED INTERVALS

In [ ]:
def calculate_statistical_intervals(forecasts, residuals, confidence_level=0.80):
    """
    Calculate prediction intervals based on residual distribution
    Assumes residuals are approximately normal (or use quantiles)
    """
    
    # Calculate residual standard deviation
    residual_std = residuals.std()
    residual_mean = residuals.mean()
    
    # For 80% confidence, use z-score for 2-tailed test
    z_score = stats.norm.ppf((1 + confidence_level) / 2)
    
    # Margin of error
    margin = z_score * residual_std
    
    # Intervals
    lower_bound = forecasts - margin
    upper_bound = forecasts + margin
    
    return lower_bound, upper_bound, margin

# Example: Calculate intervals for a model
if 'residual' not in cv_forecasts.columns:
    cv_forecasts['residual'] = cv_forecasts['actual'] - cv_forecasts['forecast']

# Get residuals from champion model
champion_residuals = cv_forecasts[cv_forecasts['model_name'] == cv_forecasts['model_name'].iloc[0]]['residual']
champion_forecasts = cv_forecasts[cv_forecasts['model_name'] == cv_forecasts['model_name'].iloc[0]]['forecast']

lower, upper, margin = calculate_statistical_intervals(champion_forecasts.values, champion_residuals.values, 0.80)

print(f"\nPrediction Interval Example (80% confidence):")
print(f"  Margin of error: ±{margin.mean():.2f} units")
print(f"  Lower bound: {lower[:5]}")
print(f"  Upper bound: {upper[:5]}")

In [ ]:
def visualize_prediction_intervals(actual, forecast, lower, upper, title='Prediction Intervals'):
    """
    Visualize forecasts with prediction intervals
    """
    fig, ax = plt.subplots(figsize=(14, 6))
    
    # Plot actual
    ax.plot(range(len(actual)), actual, marker='o', label='Actual', color='black', linewidth=2)
    
    # Plot forecast
    ax.plot(range(len(forecast)), forecast, marker='s', label='Point Forecast', 
            color='blue', linewidth=1.5, alpha=0.7)
    
    # Plot interval as shaded region
    ax.fill_between(range(len(forecast)), lower, upper, 
                     alpha=0.3, color='blue', label='80% Prediction Interval')
    
    ax.set_xlabel('Time Period')
    ax.set_ylabel('Demand')
    ax.set_title(title)
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Visualize example
sample_size = min(100, len(cv_forecasts))
sample_actual = cv_forecasts['actual'].iloc[:sample_size].values
sample_forecast = cv_forecasts['forecast'].iloc[:sample_size].values
sample_lower = lower[:sample_size]
sample_upper = upper[:sample_size]

visualize_prediction_intervals(sample_actual, sample_forecast, sample_lower, sample_upper)

---
## SECTION 3: CONFORMAL PREDICTION

### What Is Conformal Prediction?

Statistical intervals assume residuals are normally distributed and constant over time.

Real forecasts don't always follow that assumption.

**Conformal prediction** is a model-agnostic method that:
1. Doesn't assume any specific distribution
2. Calibrates intervals to actual observed coverage
3. Adapts to the real world, not theory

In [ ]:
def conformal_prediction_intervals(forecasts, residuals, confidence_level=0.80, method='quantile'):
    """
    Calibrate prediction intervals using conformal prediction
    Uses observed quantiles of residuals rather than theoretical distribution
    """
    
    if method == 'quantile':
        # Use quantiles of observed residuals
        alpha = 1 - confidence_level  # Two-tailed, so split alpha
        lower_quantile = alpha / 2
        upper_quantile = 1 - (alpha / 2)
        
        lower_margin = residuals.quantile(lower_quantile)
        upper_margin = residuals.quantile(upper_quantile)
        
        lower_bound = forecasts + lower_margin
        upper_bound = forecasts + upper_margin
    
    return lower_bound, upper_bound

# Calculate conformal intervals
if 'residual' in cv_forecasts.columns:
    conf_residuals = cv_forecasts['residual'].dropna()
    conf_forecasts = cv_forecasts['forecast'].iloc[:len(conf_residuals)].values
    
    conf_lower, conf_upper = conformal_prediction_intervals(
        conf_forecasts, conf_residuals, confidence_level=0.80
    )
    
    print(f"\nConformal Prediction Intervals (80% confidence):")
    print(f"  Lower margin (from residuals): {conf_residuals.quantile(0.1):.2f}")
    print(f"  Upper margin (from residuals): {conf_residuals.quantile(0.9):.2f}")
    print(f"  Interval width (avg): {(conf_upper - conf_lower).mean():.2f} units")

In [ ]:
# Compare statistical vs conformal intervals
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Statistical intervals
axes[0].plot(range(50), sample_actual[:50], marker='o', label='Actual', color='black')
axes[0].plot(range(50), sample_forecast[:50], marker='s', label='Forecast')
axes[0].fill_between(range(50), sample_lower[:50], sample_upper[:50], 
                      alpha=0.3, label='Statistical Interval')
axes[0].set_title('Statistical Intervals (Assumes Normal Distribution)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Conformal intervals
if 'conf_lower' in locals():
    axes[1].plot(range(50), sample_actual[:50], marker='o', label='Actual', color='black')
    axes[1].plot(range(50), sample_forecast[:50], marker='s', label='Forecast')
    axes[1].fill_between(range(50), conf_lower[:50], conf_upper[:50], 
                          alpha=0.3, color='green', label='Conformal Interval')
    axes[1].set_title('Conformal Intervals (Data-Driven Calibration)')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## SECTION 4: DECISION SUPPORT WITH INTERVALS

In [ ]:
def calculate_safety_stock(forecast, lower_bound, service_level=0.95):
    """
    Use prediction intervals to set safety stock
    """
    # Safety stock = difference between upper bound and point forecast
    # This ensures we cover variations up to our confidence level
    safety_stock = upper_bound - forecast
    return safety_stock

def inventory_decisions_with_intervals(forecast, lower, upper):
    """
    Make inventory decisions using point forecast + intervals
    """
    decisions = {
        'order_point': forecast,  # Order the point forecast
        'min_inventory': lower,   # Safety stock covers downside
        'max_inventory': upper,   # Upper bound for upside risk
        'safety_stock': upper - forecast  # Buffer for uncertainty
    }
    return decisions

# Example: Inventory decisions for first SKU
sku_forecasts = cv_forecasts.iloc[:50]
decisions_df = pd.DataFrame({
    'order_point': sku_forecasts['forecast'].values,
    'min_stock': conf_lower[:50] if 'conf_lower' in locals() else sample_lower[:50],
    'max_stock': conf_upper[:50] if 'conf_upper' in locals() else sample_upper[:50],
    'safety_stock': (conf_upper[:50] if 'conf_upper' in locals() else sample_upper[:50]) - sku_forecasts['forecast'].values
})

print("\nInventory Decisions Using Prediction Intervals:")
print(decisions_df.head(10).to_string(index=False))

---
## SECTION 5: DELIVERABLE - INTERVAL SUMMARY

In [ ]:
# Create intervals dataframe for all forecasts
intervals_output = pd.DataFrame({
    'forecast': cv_forecasts['forecast'].values,
    'lower_80': conf_lower if 'conf_lower' in locals() else lower,
    'upper_80': conf_upper if 'conf_upper' in locals() else upper,
    'method': 'Conformal' if 'conf_lower' in locals() else 'Statistical',
    'coverage_target': 0.80,
    'actual': cv_forecasts['actual'].values
})

# Check coverage
intervals_output['within_interval'] = (
    (intervals_output['actual'] >= intervals_output['lower_80']) & 
    (intervals_output['actual'] <= intervals_output['upper_80'])
)

actual_coverage = intervals_output['within_interval'].mean()
print(f"\nPrediction Interval Coverage Check:")
print(f"  Target: 80%")
print(f"  Actual: {actual_coverage:.1%}")
print(f"  Status: {'✓ PASS' if 0.75 < actual_coverage < 0.85 else '⚠ NEEDS CALIBRATION'}")

# Save intervals
intervals_output.to_csv('prediction_intervals_80pct.csv', index=False)
print("\n✓ Prediction intervals saved to 'prediction_intervals_80pct.csv'")